In [87]:
#from flask import Flask, render_template, request
from bs4 import BeautifulSoup
import requests
import csv
import time
import os
from selenium import webdriver
import datetime
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [88]:

def Extract_card(card,path):
    try:
        content = card.find_element('xpath',path)
        return content
    except:
        return 0

# Create a new column 'maxunitsold' based on conditions
def calculate_maxunitsold(unitsold):
    if unitsold == 0:
        return 0
    elif unitsold < 100:
        return unitsold + 50
    elif unitsold < 1000:
        return unitsold + 100
    elif unitsold < 10000:
        return unitsold + 1000
    else:
        return unitsold * 2
    


def Scrape_Page(driver,inp_url):

    driver.get(inp_url)
    cards = driver.find_elements('xpath','//div[@data-cy="asin-faceout-container"]')
    Image_url = []
    Product_Title = []
    Quantity_sold = []
    No_of_ratings = []
    Selling_price = []
    Unit_price = []
    MRP_price = []
    Prod_url = []
    Asin = []

    for card in cards:

        # Extract title        
        title = Extract_card(card,'.//div[@data-cy="title-recipe"]')
        if title.text.startswith('Sponsored'):
            continue
        if title:
            Product_Title.append(title.text)
        else:
            Product_Title.append(0)


        # Extract image link
        image_link = Extract_card(card,'.//img[@class="s-image"]')
        if image_link:
            image_src = image_link.get_attribute('src')
            Image_url.append(image_src)
        else:
            Image_url.append(0)

        # Extract rating        
        rating = Extract_card(card,'.//div[@data-cy="reviews-block"]//div[@class = "a-row a-size-small"]//span[@class="a-size-base s-underline-text"]')
        if rating:
            No_of_ratings.append(rating.text.replace(",",""))
        else:
            No_of_ratings.append(0)



        # Extract Quantity        
        qty = Extract_card(card,'.//div[@data-cy="reviews-block"]//div[@class = "a-row a-size-base"][last()]')
        if qty:
            quantity = qty.text.split(" ")[0].replace('+','')
            if quantity[-1] == 'K':
                actual_value = int(quantity[:-1])*1000
            else:
                actual_value = int(quantity[:-1])*10 + int(quantity[-1])
            Quantity_sold.append(actual_value)
        else:
            Quantity_sold.append(0)


        # Extract Selling Price        
        sp = Extract_card(card,'.//div[@data-cy="price-recipe"]//div[@class="a-row a-size-base a-color-base"]//span[@class="a-price"]//span[@class="a-price-whole"]')
        if sp:
            Selling_price.append(sp.text.replace(",",""))
        else:
            Selling_price.append(0)


        # Extract Unit Price per 100ml        
        up = Extract_card(card,'.//div[@data-cy="price-recipe"]//div[@class="a-row a-size-base a-color-base"]//span[@class="a-size-base a-color-secondary"]//span[@class="a-price a-text-price"]')
        if up:
            Unit_price.append(up.text[1:].replace(",",""))
        else:
            Unit_price.append(0)



        # Extract MRP Price        
        mrp = Extract_card(card,'.//div[@data-cy="price-recipe"]//div[@class="a-row a-size-base a-color-base"]//div[@class="a-section aok-inline-block"]//span[@class="a-price a-text-price"]')
        if mrp:
            MRP_price.append(mrp.text[1:].replace(",",""))
        else:
            MRP_price.append(0)

        # Extract Product Link and ASIN Number        
        prod_link = Extract_card(card,'.//div[@data-cy="title-recipe"]//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        if prod_link:
            link = prod_link.get_attribute('href')
            Prod_url.append(link.split('=')[0])

            try:
                url_parts = link.split("/")
                dp_index = url_parts.index("dp")
                asin = url_parts[dp_index + 1]
                Asin.append(asin)

            except:
                Asin.append(0)

        else:
            Prod_url.append(0)

    return Image_url,Product_Title,Quantity_sold,No_of_ratings,Selling_price,Unit_price,MRP_price,Prod_url,Asin
    
    
    


In [89]:
def Extract_table(drive,path):
    try:
        content = drive.find_element('xpath',path)
        return content.text.strip()
    except:
        return 0


def Brand_Scrape(df):
    time_interval = 2

    driver = webdriver.Firefox()
    brands = []
    urls = []
    size = []
    brand_value = []
    scent = []
    skin_type = []
    benefits = []
    item_form = []
    item_weight = []
    active_ingredient = []
    unit_count = []
    skin_tone = []
    item_volume = []
    special_feature = []
    pack_size = []
    spf_factor = []
    hair_type = []
    material_type_free = []
    recomended_for = []
    prefixes = ["Visit the ", "Brand: "]
    for index, row in df.iterrows():
        amazon_url = row['Product_URL']
        driver.get(amazon_url)

        brand_name = Extract_table(driver,'//*[@id="bylineInfo"]')
        if brand_name:
            if brand_name.startswith("Visit the "):
                brand_name = brand_name[len("Visit the "):]
            if brand_name.startswith("Brand: "):
                brand_name = brand_name[len("Brand: "):]
            if brand_name.endswith("Store"):
                brand_name = brand_name[:-5]
        brands.append(brand_name)
        urls.append(amazon_url)


        ml = Extract_table(driver,'//*[@id="variation_size_name"]//span[@class="selection"]')
        size.append(ml)
             
        bv = Extract_table(driver,'//tr[@class= "a-spacing-small po-brand"]//td[2]')
        brand_value.append(bv)
        
        sc = Extract_table(driver,'//tr[@class= "a-spacing-small po-scent"]//td[2]')
        scent.append(sc)
        
        sty = Extract_table(driver,'//tr[@class= "a-spacing-small po-skin_type"]//td[2]')
        skin_type.append(sty)
        
        bf = Extract_table(driver,'//tr[@class= "a-spacing-small po-product_benefit"]//td[2]')
        benefits.append(bf)
        
        itf = Extract_table(driver,'//tr[@class= "a-spacing-small po-item_form"]//td[2]')
        item_form.append(itf)
        
        iw = Extract_table(driver,'//tr[@class= "a-spacing-small po-item_weight"]//td[2]')
        item_weight.append(iw)
        
        ai = Extract_table(driver,'//tr[@class= "a-spacing-small po-active_ingredients"]//td[2]')
        active_ingredient.append(ai)
        
        uc = Extract_table(driver,'//tr[@class= "a-spacing-small po-unit_count"]//td[2]')
        unit_count.append(uc)
        
        st = Extract_table(driver,'//tr[@class= "a-spacing-small po-skin_tone"]//td[2]')
        skin_tone.append(st)
        
        iv = Extract_table(driver,'//tr[@class= "a-spacing-small po-item_volume"]//td[2]')
        item_volume.append(iv)
        
        ps = Extract_table(driver,'//tr[@class= "a-spacing-small po-number_of_items"]//td[2]')
        pack_size.append(ps)
        
        spf = Extract_table(driver,'//tr[@class= "a-spacing-small po-sun_protection_factor"]//td[2]')
        spf_factor.append(spf)
        
        ht = Extract_table(driver,'//tr[@class= "a-spacing-small po-hair_type"]//td[2]')
        hair_type.append(ht)
        
        mt = Extract_table(driver,'//tr[@class= "a-spacing-small po-material_type_free"]//td[2]')
        material_type_free.append(mt)
        
        sf = Extract_table(driver,'//tr[@class= "a-spacing-small po-special_features"]//td[2]')
        special_feature.append(sf)
        
        rf = Extract_table(driver,'//tr[@class= "a-spacing-small po-recommended_uses_for_product"]//td[2]')
        recomended_for.append(rf)        
            
        time.sleep(time_interval)
    driver.close()

    op_df = pd.DataFrame({
'AZ_URL':urls,
'Size_of_SKU':size,
'Brand_Name':brands,
'Brand_value':brand_value,
'Scent_type':scent,
'Skin_type':skin_type,
'Benefits':benefits,
'Item_form':item_form,
'Item_weight':item_weight,
'Active_ingredient':active_ingredient,
'Unit_Count':unit_count,
'Skin_tone':skin_tone,
'Item_volume':item_volume,
'Special_feature':special_feature,
'Pack_size':pack_size,
'SPF_factor':spf_factor,
'Hair_type':hair_type,
'Material_type_free':material_type_free,
'Special_feature':special_feature,
'Recomended_for':recomended_for,})
    
    return op_df



In [90]:
def Find_competitor(df):
    
    df_grouped = df.groupby('Brand_Name')['Revenue'].sum().reset_index()
    df_sorted = df_grouped.sort_values(by='Revenue', ascending=False)
    top_10_brands = df_sorted.head(10)['Brand_Name'].tolist()
    
    return top_10_brands

In [91]:

time_interval = 2

# Define how many page to scrape
num_of_pages = 1
start_time = datetime.datetime.now()
print('Card Scraping started at',start_time)
print('Hold your seat belt tight and enjoy the show !!!....')

categories = ['Face wash','Face Serum','Face Moisturizers','Shampoo','Conditioner','Hair Growth Serum',
              'Sunscreen','Hair growth Oil','Body Lotion','Hair Mask','Facial Cleanser','Body Mist','EDP - Women','EDP - Men']

keyword ="Shampoo"

category_link = f'https://www.amazon.in/s?k={keyword.replace(" ","+")}&s=exact-aware-popularity-rank&page='

Image_url_m = []
Product_Title_m = []
Quantity_sold_m = []
No_of_ratings_m = [] 
Selling_price_m = []
Unit_price_m = []
MRP_price_m = []
Prod_url_m = []
Asin_m = []

Top_brands = []

driver = webdriver.Firefox()
for i in range(1,num_of_pages+1):
    url = category_link+str(i)
    Image_url,Product_Title,Quantity_sold,No_of_ratings,Selling_price,Unit_price,MRP_price,Prod_url,Asin = Scrape_Page(driver,url)
    Image_url_m.extend(Image_url)
    Product_Title_m.extend(Product_Title)
    Quantity_sold_m.extend(Quantity_sold)
    No_of_ratings_m .extend(No_of_ratings)
    Selling_price_m.extend(Selling_price)
    Unit_price_m.extend(Unit_price)
    MRP_price_m.extend(MRP_price)
    Prod_url_m.extend(Prod_url)
    Asin_m.extend(Asin)
    time.sleep(time_interval)
    
driver.close()
    
df = pd.DataFrame({'Product_Title':Product_Title_m, 'ASIN':Asin_m,'Product_URL':Prod_url_m,'Image_URL':Image_url_m,
                   'No_Of_Ratings':No_of_ratings_m,'MRP_Price':MRP_price_m,'Per_100ml_price':Unit_price_m,
                   'Selling_Price':Selling_price_m,'Unit_Sold':Quantity_sold_m})
# Drop the duplicates
df_unique = df.drop_duplicates(subset='ASIN')

# Calculate the max unit slold
df_unique['Max_Unit_Sold'] = df_unique['Unit_Sold'].apply(calculate_maxunitsold)

df_unique[['No_Of_Ratings', 'MRP_Price','Selling_Price']] = df_unique[['No_Of_Ratings', 'MRP_Price','Selling_Price']].astype(int)
df_unique[['Per_100ml_price']] = df_unique[['Per_100ml_price']].astype(float)
 

# Calculate the revenue
df_unique['Revenue'] = df_unique['Unit_Sold'] * df_unique['Selling_Price']
df_unique['Max_Revenue'] = df_unique['Max_Unit_Sold'] * df_unique['Selling_Price']


# Calculate the contribution of each product
total_revenue = df_unique['Revenue'].sum()
df_unique['Contribution'] = (df_unique['Revenue'] / total_revenue)


# Sort the DataFrame by contribution in descending order
df_unique = df_unique.sort_values('Contribution', ascending=False)
df_unique['Rolling_sum'] = df_unique['Contribution'].cumsum()


end_time = datetime.datetime.now()
card_time_taken = round((end_time - start_time).total_seconds() / 60,1)
print(f'Card Scraping done in : {card_time_taken} mins')
# df_unique.to_csv(f'{keyword}_MS_without_brand.csv',index=False)


print('Brand Scraping Started....',end_time)
brand_df = Brand_Scrape(df_unique)

final_op = pd.merge(df_unique, brand_df, how='left', left_on='Product_URL', right_on='AZ_URL')

final_op.drop(columns=['AZ_URL'], inplace=True)
final_op['Category'] = keyword

# exporting the competitor
Top_brands = Find_competitor(final_op)
Competitor = final_op[final_op['Brand_Name'].isin(Top_brands)]
Competitor.to_csv(f'{keyword}_Competitor_of_Pilgrim.csv',index=False)

# Exporting Pilgrim's product
Pilgrim = final_op[final_op['Brand_Name'] == 'Pilgrim']
Pilgrim.to_csv(f'{keyword}_Products_of_Pilgrim.csv',index=False)




brand_time_taken = round((datetime.datetime.now() - end_time).total_seconds() / 60,1)
print(f'Brand name scraping done in : {brand_time_taken} mins')

final_op.to_csv(f'{keyword}_MS_with_brand.csv',index=False)
print("Congrats!!! Market Sizing Done")

Card Scraping started at 2024-09-27 20:24:43.511114
Hold your seat belt tight and enjoy the show !!!....
Card Scraping done in : 0.3 mins
Brand Scraping Started.... 2024-09-27 20:25:01.752834
Brand name scraping done in : 4.7 mins
Congrats!!! Market Sizing Done
